In [1]:
  
index_schema = {
          "tag": [
            {
              "name": "document_type"
            }
        
          ],
          "text": [    {
              "name": "major"
            },
            {
              "name": "institute"
            },
            {
              "name": "study_field"
            },
            {
              "name": "major_level"
            },
            {
              "name": "major_type"
            },
            {
              "name": "location"
            },
            {
              "name": "language"
            },
            {
              "name": "major_duration"
            },
            {
              "name": "major_field"
            },
            {
              "name": "financing_type"
            },
            {
              "name": "content"
            },
            {
              "name": "source"
            }
          ],
          "vector": [
            {
              "name": "content_vector",
              "algorithm": "FLAT",
              "block_size": 1000,
              "datatype": "FLOAT32",
              "dims": 1536,
              "distance_metric": "COSINE",
              "initial_cap": 20000
            }
          ]
        }

In [50]:
from langchain_community.vectorstores.redis import RedisTag, RedisNum,RedisText
from langchain_openai import (
    OpenAI,
    AzureOpenAI,
    ChatOpenAI,
    AzureChatOpenAI,
    OpenAIEmbeddings,
    AzureOpenAIEmbeddings,
)

institute = RedisText("institute") % "Pannon"
major = RedisTag("major") == "pszichológia"
filter = institute & major
print(str(filter))

filter = '(@institute:%%%Széchenyi%%% | @institute:*Széchenyi*) (@major:%%%informatikus%%% | @major:*informatikus*)'

print(str(filter))

(@institute:(Pannon) @major:{pszichológia})
(@institute:%%%Széchenyi%%% | @institute:*Széchenyi*) (@major:%%%informatikus%%% | @major:*informatikus*)


In [51]:
from langchain_community.vectorstores.redis import Redis
import os


REDIS_URL_PREFIX = f"redis://default:@localhost:6380/"

embedder = AzureOpenAIEmbeddings(
                    azure_endpoint="https://chatboss-test-openai.openai.azure.com/",
                    openai_api_version="2023-09-01-preview",
                    azure_deployment="chatboss_sweden-central_embedding_text-embedding-3-large_1",
                    api_key = "1qP4h5YKQglreWwtufsBnKoGe6MyHDl05a9dx0YPHrOsSWtTLCZKJQQJ99AKACfhMk5XJ3w3AAABACOG4Bbo"
                )

vector_store = Redis.from_existing_index(
                embedding=embedder,
                index_name="test_calculation",
                schema=index_schema,
                redis_url=REDIS_URL_PREFIX + "0",
            )

retriever = vector_store.as_retriever(
                    search_kwargs= {
                        "k": 100,
                        "lambda_mult": 0.5,
                        "filter": filter,
                    }
                )


docs = retriever.invoke("Milyen tárgyakból kell érettségiznem a Pannon Egyetem pszichológia szakára?",filter=filter)






Metadata key major_duration not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['major', 'institute', 'study_field', 'major_level', 'major_type', 'location', 'language', 'major_duration', 'major_field', 'financing_type', 'source', 'document_type']
Metadata key major_field not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['major', 'institute', 'study_field', 'major_level', 'major_type', 'location', 'language', 'major_duration', 'major_field', 'financing_type', 'source', 'document_type']
Metadata key major_duration not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['major', 'institute', 'study_field', 'major_level', 'major_type', 'location', 'language', 'major_duration', 'major_field', 'financing_type', 'source', 'document_type']
Metadata key major_field not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['major', 'institute', 'study_field', 'major_level'

In [52]:
for doc in docs:
    print(doc.metadata)

{'id': 'doc:test_calculation:be7cd6e38f664403a8de309d1cae6237', 'major': 'gazdaságinformatikus (A)', 'institute': 'Széchenyi István Egyetem (SZE)', 'study_field': 'informatika', 'major_level': 'alapképzés (BA/BSc/BProf)', 'major_type': 'nappali', 'location': 'Győr', 'language': 'angol', 'major_duration': None, 'major_field': None, 'financing_type': 'önköltséges', 'source': 'https://www.felvi.hu/felveteli/meghirdetesek_a/!FFT_Megjelenito_A/intezmenyek/35/kepzesek/40051', 'document_type': 'pontszámítás'}
{'id': 'doc:test_calculation:de3f93fcbbbe4af48bd907466be47532', 'major': 'gazdaságinformatikus (A)', 'institute': 'Széchenyi István Egyetem (SZE)', 'study_field': 'informatika', 'major_level': 'alapképzés (BA/BSc/BProf)', 'major_type': 'nappali', 'location': 'Győr', 'language': 'magyar', 'major_duration': None, 'major_field': None, 'financing_type': 'önköltséges', 'source': 'https://www.felvi.hu/felveteli/meghirdetesek_a/!FFT_Megjelenito_A/intezmenyek/35/kepzesek/24556', 'document_type':

In [156]:
vector_store.similarity_search(
    "Milyen tárgyakból kell érettségiznem a Pannon Egyetem pszichológia szakára?",
                               k=5,
                               filter=filter
                               )

[]